##### Imports

In [42]:
# %pip install -r requirements.txt

In [43]:
import pandas as pd
import numpy as np
from urllib.request import urlopen
import certifi
import json
from fredapi import Fred
import os
import ssl

# Custom packages
import derive_data as dd

import warnings
warnings.filterwarnings("ignore")

# Environment variables
import dotenv
dotenv.load_dotenv()
FRED_API_KEY = os.getenv("FRED_API_KEY")
FMP_API_KEY = os.getenv("FMP_API_KEY")

# Data Collection

### Inputs

In [44]:
def get_jsonparsed_data(url):
    context = ssl.create_default_context(cafile=certifi.where())
    response = urlopen(url, context=context)
    # response = urlopen(url, cafile=certifi.where())
    data = response.read().decode("utf-8")
    return json.loads(data)

idx = pd.read_csv('data/inputs/index_symbols.csv')
comm = pd.read_csv('data/inputs/commodity_symbols.csv')

url = f"https://financialmodelingprep.com/stable/index-list?apikey={FMP_API_KEY}"
fmp_idx = pd.DataFrame(get_jsonparsed_data(url))
fmp_idx = fmp_idx[fmp_idx['symbol'].isin(idx['FMP API Symbol'])].reset_index(drop=True)
fmp_idx['fx_symbol'] = fmp_idx['currency'].apply(lambda x: x+'USD' if x != 'USD' else None)

url = f"https://financialmodelingprep.com/stable/commodities-list?apikey={FMP_API_KEY}"
fmp_comm = pd.DataFrame(get_jsonparsed_data(url))
fmp_comm = fmp_comm[fmp_comm['symbol'].isin(comm['FMP API Symbol'])].reset_index(drop=True)

fmp_idx.to_csv('data/inputs/fmp_index_list.csv', index=False)
fmp_comm.to_csv('data/inputs/fmp_commodity_list.csv', index=False)

date_from = '1990-01-01'
date_to = '2025-10-24'

### Equity Index, Commodity, and FX Daily Timeseries data 

In [45]:
# symbol = fmp_idx.loc[0, 'symbol']
# url = f"https://financialmodelingprep.com/stable/historical-price-eod/full?symbol={symbol}&from={date_from}&to={date_to}&apikey={FMP_API_KEY}"
# df = pd.DataFrame(get_jsonparsed_data(url))\
#     [['symbol', 'date', 'open', 'high', 'low', 'close', 'volume', 'vwap']]
# df['date'] = pd.to_datetime(df['date'])
# df = df.set_index('date')
# df.columns = pd.MultiIndex.from_product([[df['symbol'].iloc[0]], df.columns])
# df = df.drop(columns=df.columns[0]).sort_index()

# for i in range(1, len(fmp_idx)):
#     symbol = fmp_idx.loc[i, 'symbol']
#     url = f"https://financialmodelingprep.com/stable/historical-price-eod/full?symbol={symbol}&from={date_from}&to={date_to}&apikey={FMP_API_KEY}"
#     temp = pd.DataFrame(get_jsonparsed_data(url))\
#         [['symbol', 'date', 'open', 'high', 'low', 'close', 'volume', 'vwap']]
#     temp['date'] = pd.to_datetime(temp['date'])
#     temp = temp.set_index('date')
#     temp.columns = pd.MultiIndex.from_product([[temp['symbol'].iloc[0]], temp.columns])
#     temp = temp.drop(columns=temp.columns[0]).sort_index()
#     df = pd.concat([df, temp], axis=1)

# msci = pd.read_excel('data/inputs/MSCI_China_Index.xlsx')[:-1]
# msci['Date'] = pd.to_datetime(msci['Date'])
# # msci = msci.reindex(index=df.index)
# msci = msci.set_index('Date')
# msci.columns = pd.MultiIndex.from_product([['MSCI_China'], ['close']])
# msci[('MSCI_China', 'volume')] = np.nan

# df.join(msci, how='left').to_csv('data/processed/index_data.csv')

# ## -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------###

# symbol = fmp_comm.loc[0, 'symbol']
# url = f"https://financialmodelingprep.com/stable/historical-price-eod/full?symbol={symbol}&from={date_from}&to={date_to}&apikey={FMP_API_KEY}"
# df = pd.DataFrame(get_jsonparsed_data(url))\
#     [['symbol', 'date', 'open', 'high', 'low', 'close', 'volume', 'vwap']]
# df['date'] = pd.to_datetime(df['date'])
# df = df.set_index('date')
# df.columns = pd.MultiIndex.from_product([[df['symbol'].iloc[0]], df.columns])
# df = df.drop(columns=df.columns[0]).sort_index()

# for i in range(1, len(fmp_comm)):
#     symbol = fmp_comm.loc[i, 'symbol']
#     url = f"https://financialmodelingprep.com/stable/historical-price-eod/full?symbol={symbol}&from={date_from}&to={date_to}&apikey={FMP_API_KEY}"
#     temp = pd.DataFrame(get_jsonparsed_data(url))\
#         [['symbol', 'date', 'open', 'high', 'low', 'close', 'volume', 'vwap']]
#     temp['date'] = pd.to_datetime(temp['date'])
#     temp = temp.set_index('date')
#     temp.columns = pd.MultiIndex.from_product([[temp['symbol'].iloc[0]], temp.columns])
#     temp = temp.drop(columns=temp.columns[0]).sort_index()
#     df = pd.concat([df, temp], axis=1)

# # df.to_csv('data/commodity_data.csv')
# nickel = pd.read_csv('data/inputs/Nickel_futures.csv', parse_dates=['Date'], dayfirst=True, index_col='Date')\
#     .rename_axis('date').sort_index().rename(columns={'Price': 'close', 'Vol.': 'volume', 'Open': 'open', 'High': 'high', 'Low': 'low'})\
#         [['open', 'high', 'low', 'close', 'volume']]
# nickel.index.name = 'date'
# for col in nickel.columns:
#     if nickel[col].dtype == 'object':
#         nickel[col] = pd.to_numeric(nickel[col].astype(str).str.replace(',', ''), errors='coerce')
# nickel.columns = pd.MultiIndex.from_product([['Nickel'], nickel.columns])
# nickel[('Nickel', 'volume')] = np.nan
# df.join(nickel, how='left').to_csv('data/processed/commodity_data.csv')

# ## -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------###

# fx_symbols_retieved = []
# symbol = fmp_idx.loc[0, 'fx_symbol']
# fx_symbols_retieved.append(symbol)
# url = f"https://financialmodelingprep.com/stable/historical-price-eod/full?symbol={symbol}&from={date_from}&to={date_to}&apikey={FMP_API_KEY}"
# df = pd.DataFrame(get_jsonparsed_data(url))[['symbol', 'date', 'close']]
# df['date'] = pd.to_datetime(df['date'])
# df = df.set_index('date')
# df.rename(columns={'close': df['symbol'].iloc[0]}, inplace=True)
# df = df.drop(columns=df.columns[0]).sort_index()

# for i in range(1, len(fmp_idx)):
#     symbol = fmp_idx.loc[i, 'fx_symbol']
#     if symbol is None or symbol in fx_symbols_retieved:
#         continue
#     fx_symbols_retieved.append(symbol)
#     url = f"https://financialmodelingprep.com/stable/historical-price-eod/full?symbol={symbol}&from={date_from}&to={date_to}&apikey={FMP_API_KEY}"
#     temp = pd.DataFrame(get_jsonparsed_data(url))[['symbol', 'date', 'close']]
#     temp['date'] = pd.to_datetime(temp['date'])
#     temp = temp.set_index('date')
#     temp.rename(columns={'close': temp['symbol'].iloc[0]}, inplace=True)
#     temp = temp.drop(columns=temp.columns[0]).sort_index()
#     df = pd.concat([df, temp], axis=1)

# df.to_csv('data/processed/fx_data.csv')

### Macroeconomic Data

In [46]:
# Core macro and rate series
macro_codes_dict = {
    # RATE BENCHMARKS
    'FEDFUNDS': 'Federal Funds Effective Rate',
    'SOFR': 'Secured Overnight Financing Rate',
    # TREASURY RATES
    'GS1M': '1-Month Treasury Rate',
    'GS3M': '3-Month Treasury Rate', 
    'GS6M': '6-Month Treasury Rate',
    'GS1': '1-Year Treasury Rate',
    'GS2': '2-Year Treasury Rate',
    'GS3': '3-Year Treasury Rate',
    'GS5': '5-Year Treasury Rate',
    'GS7': '7-Year Treasury Rate',
    'GS10': '10-Year Treasury Rate',
    'GS20': '20-Year Treasury Rate',
    'GS30': '30-Year Treasury Rate',
    # OTHER RATES
    'AAA': 'Moody\'s Seasoned AAA Corporate Bond Yield',
    'BAA': 'Moody\'s Seasoned BAA Corporate Bond Yield',
    'MORTGAGE30US': '30-Year Fixed Rate Mortgage Average',
    'DPRIME': 'Bank Prime Loan Rate',
    'T5YIE': '5-Year Breakeven Inflation Rate',
    'T10YIE': '10-Year Breakeven Inflation Rate',
    'T30YIE': '30-Year Breakeven Inflation Rate',
    # ECONOMIC INDICATORS
    'GDP': 'Gross Domestic Product',
    'GDPC1': 'Real Gross Domestic Product',
    'A939RX0Q048SBEA': 'Real GDP Per Capita',
    'PCE': 'Personal Consumption Expenditures',
    'PCEPI': 'Personal Consumption Expenditures Price Index',
    'PCEC96': 'Real Personal Consumption Expenditures',
    'CPIAUCSL': 'Consumer Price Index',
    'CPILFESL': 'Core CPI (Less Food and Energy)',
    'UNRATE': 'Unemployment Rate',
    'CIVPART': 'Labor Force Participation Rate',
    'INDPRO': 'Industrial Production Index',
    'PAYEMS': 'Total Nonfarm Payrolls',
    'HOUST': 'Housing Starts',
    'PERMIT': 'Building Permits',
    'MTSDS133FMS': 'Monthly US Government Surplus/Deficit',
    'GFDEGDQ188S': 'Federal Government Debt to GDP Ratio',
    'PMSAVE': 'Personal Savings',
    'PSAVERT': 'Personal Saving Rate',
    'GPDI': 'Gross Private Domestic Investment',
    'GPDIC1': 'Real Gross Private Domestic Investment',
    'BOGZ1FU263092001Q': 'Foreign Direct Investment in the United States',
    'QBPBSTAS': 'Balance Sheet - Total Assets',
    'FDHBFRBN': 'Federal Debt Held by Federal Reserve Banks',
    'FYGFDPUN': 'Federal Debt Held by the Public',
    'FDHBFIN': 'Federal Debt Held by Foreign Investors',
    'COMPOUT': 'Commercial Paper Outstanding',
    'ABCOMP': 'Asset-Backed Commercial Paper Outstanding',
    # MONEY SUPPLY
    'M1SL': 'M1 Money Stock',
    'M2SL': 'M2 Money Stock',
    'BASE': 'St. Louis Adjusted Monetary Base',
    # MARKET INDICATORS
    'VIXCLS': 'CBOE Volatility Index (VIX)',
    'UMCSENT': 'University of Michigan Consumer Sentiment',
    'USSLIND': 'Leading Index for the United States',
    'VISASMIHSA': 'Visa U.S. Consumer Spending Momentum Index: Headline',
    'VISASMIDSA': 'Visa U.S. Consumer Spending Momentum Index: Discretionary',
}

macro_units = {
    # INTEREST RATES AND FINANCIAL RATES
    'FEDFUNDS': 'Percent, Seasonally Adjusted',
    'SOFR': 'Percent, Not Seasonally Adjusted',
    'GS1M': 'Percent, Not Seasonally Adjusted',
    'GS3M': 'Percent, Not Seasonally Adjusted',
    'GS6M': 'Percent, Not Seasonally Adjusted',
    'GS1': 'Percent, Not Seasonally Adjusted',
    'GS2': 'Percent, Not Seasonally Adjusted',
    'GS3': 'Percent, Not Seasonally Adjusted',
    'GS5': 'Percent, Not Seasonally Adjusted',
    'GS7': 'Percent, Not Seasonally Adjusted',
    'GS10': 'Percent, Not Seasonally Adjusted',
    'GS20': 'Percent, Not Seasonally Adjusted',
    'GS30': 'Percent, Not Seasonally Adjusted',
    'AAA': 'Percent, Not Seasonally Adjusted',
    'BAA': 'Percent, Not Seasonally Adjusted',
    'MORTGAGE30US': 'Percent, Not Seasonally Adjusted',
    'DPRIME': 'Percent, Not Seasonally Adjusted',
    'T5YIE': 'Percent, Not Seasonally Adjusted',
    'T10YIE': 'Percent, Not Seasonally Adjusted',
    'T30YIE': 'Percent, Not Seasonally Adjusted',
    # ECONOMIC INDICATORS
    'GDP': 'Billions of Dollars, Seasonally Adjusted Annual Rate',
    'GDPC1': 'Billions of Chained 2017 Dollars, Seasonally Adjusted Annual Rate',
    'A939RX0Q048SBEA': 'Chained 2017 Dollars, Seasonally Adjusted',
    'PCE': 'Billions of Dollars, Seasonally Adjusted Annual Rate',
    'PCEPI': 'Index 2017=100, Seasonally Adjusted',
    'PCEC96': 'Billions of Chained 2017 Dollars, Seasonally Adjusted',
    'CPIAUCSL': 'Index 1982-1984=100, Seasonally Adjusted',
    'CPILFESL': 'Index 1982-1984=100, Seasonally Adjusted',
    'UNRATE': 'Percent, Seasonally Adjusted',
    'CIVPART': 'Percent, Seasonally Adjusted',
    'INDPRO': 'Index 2017=100, Seasonally Adjusted',
    'PAYEMS': 'Thousands of Persons, Seasonally Adjusted',
    'HOUST': 'Thousands of Units, Seasonally Adjusted Annual Rate',
    'PERMIT': 'Thousands of Units, Seasonally Adjusted Annual Rate',
    'MTSDS133FMS': 'Millions of Dollars, Not Seasonally Adjusted',
    'GFDEGDQ188S': 'Percent of GDP, Not Seasonally Adjusted',
    'PMSAVE': 'Billions of Dollars, Not Seasonally Adjusted',
    'PSAVERT': 'Percent, Seasonally Adjusted',
    'GPDI': 'Billions of Dollars, Seasonally Adjusted Annual Rate',
    'GPDIC1': 'Billions of Chained 2017 Dollars, Seasonally Adjusted Annual Rate',
    'BOGZ1FU263092001Q': 'Millions of Dollars, Not Seasonally Adjusted',
    'QBPBSTAS': 'Millions of Dollars, Not Seasonally Adjusted',
    'FDHBFRBN': 'Billions of Dollars, Not Seasonally Adjusted',
    'FYGFDPUN': 'Millions of Dollars, Not Seasonally Adjusted',
    'FDHBFIN': 'Billions of Dollars, Not Seasonally Adjusted',
    'COMPOUT': 'Billions of Dollars, Not Seasonally Adjusted',
    'ABCOMP': 'Billions of Dollars, Not Seasonally Adjusted',
    # MONEY SUPPLY
    'M1SL': 'Billions of Dollars, Seasonally Adjusted',
    'M2SL': 'Billions of Dollars, Seasonally Adjusted',
    'BASE': 'Millions of Dollars, Not Seasonally Adjusted',
    # MARKET INDICATORS
    'VIXCLS': 'Index, Not Seasonally Adjusted',
    'UMCSENT': 'Index 1966:Q1=100, Not Seasonally Adjusted',
    'USSLIND': 'Percent, Seasonally Adjusted',
    'VISASMIHSA': 'Index, Seasonally Adjusted',
    'VISASMIDSA': 'Index, Seasonally Adjusted',
}


In [47]:
def get_comprehensive_macro_data(fred_api_key, series_dict=macro_codes_dict, start_date='1990-01-01', end_date='2025-10-24'):
    fred = Fred(api_key=fred_api_key)
    all_data = pd.DataFrame(index=pd.DatetimeIndex(pd.date_range(start=start_date, end=end_date, freq='D')))
    successful_series = []
    failed_series = []
    
    for code, description in series_dict.items():
        try:
            if code in ['MORTGAGE30US', 'BASE']:
                series_data = fred.get_series(code, obervation_satrt=start_date, frequency='m', aggregation_method='eop').rename(code)
            else:
                series_data = fred.get_series(code, observation_start=start_date).rename(code)
            if not series_data.empty:
                all_data = all_data.join(series_data, how='left')
                successful_series.append((code, description))
                # print(f"✓ {code}: {len(series_data)} observations")
            else:
                failed_series.append((code, "No data in time range"))
                # print(f"✗ {code}: No data in specified time range")
        except Exception as e:
            failed_series.append((code, str(e)))
            # print(f"✗ {code}: {e}")

    all_data = all_data.sort_index()
    all_data.index.name = 'date'
    # print(f"Final dataset: {all_data.shape[0]} observations, {all_data.shape[1]} variables")
    # print(f"Date range: {all_data.index.min()} to {all_data.index.max()}")
    return all_data, successful_series, failed_series

macro_data, successful, failed = get_comprehensive_macro_data(FRED_API_KEY, start_date=date_from, end_date=date_to)

# Separate columns by their data frequency (daily, monthly, quarterly, etc.)
def infer_frequency(series):
    # Drop NaNs and get sorted index
    idx = series.dropna().index
    if len(idx) < 2:
        return 'unknown'
    # Calculate median difference in days
    freq_days = (idx[1:] - idx[:-1]).days
    median_days = np.median(freq_days)
    if median_days <= 2:
        return 'daily'
    elif 25 <= median_days <= 35:
        return 'monthly'
    elif 80 <= median_days <= 100:
        return 'quarterly'
    elif 350 <= median_days <= 370:
        return 'yearly'
    else:
        return 'other'

frequency_map = {}
for col in macro_data.columns:
    frequency_map[col] = infer_frequency(macro_data[col])

daily_cols = [col for col, freq in frequency_map.items() if freq == 'daily']
monthly_cols = [col for col, freq in frequency_map.items() if freq == 'monthly']
quarterly_cols = [col for col, freq in frequency_map.items() if freq == 'quarterly']
# yearly_cols = [col for col, freq in frequency_map.items() if freq == 'yearly']
# other_cols = [col for col, freq in frequency_map.items() if freq == 'other']
# unknown_cols = [col for col, freq in frequency_map.items() if freq == 'unknown']

macro_data_daily = macro_data[daily_cols].dropna(how='all', axis=0)
macro_data_monthly = macro_data[monthly_cols].dropna(how='all', axis=0)
macro_data_quarterly = macro_data[quarterly_cols].dropna(how='all', axis=0)

macro_data_daily.to_csv('data/processed/macro_data_daily.csv')
macro_data_monthly.to_csv('data/processed/macro_data_monthly.csv')
macro_data_quarterly.to_csv('data/processed/macro_data_quarterly.csv')

# Data Processing

In [48]:
fmp_idx = pd.read_csv('data/inputs/fmp_index_list.csv')
fmp_comm = pd.read_csv('data/inputs/fmp_commodity_list.csv')

symbol_name_dict = fmp_idx.set_index('symbol')['name'].to_dict()
symbol_name_dict.update(fmp_comm.set_index('symbol')['name'].to_dict())

### Read Macro Data from saved csv files

In [49]:
macro_data_daily = pd.read_csv('data/processed/macro_data_daily.csv', parse_dates=['date'], index_col='date')
macro_data_monthly = pd.read_csv('data/processed/macro_data_monthly.csv', parse_dates=['date'], index_col='date')
macro_data_quarterly = pd.read_csv('data/processed/macro_data_quarterly.csv', parse_dates=['date'], index_col='date')

macro_data_daily.columns = pd.MultiIndex.from_tuples([(col, macro_codes_dict[col], macro_units[col].split(',')[0]) for col in macro_data_daily.columns])
macro_data_monthly.columns = pd.MultiIndex.from_tuples([(col, macro_codes_dict[col], macro_units[col].split(',')[0]) for col in macro_data_monthly.columns])
macro_data_quarterly.columns = pd.MultiIndex.from_tuples([(col, macro_codes_dict[col], macro_units[col].split(',')[0]) for col in macro_data_quarterly.columns])

### Read Equity Index, Commodity, and FX data from saved csv files

In [50]:
# All commodities are in USD
commodity_data = pd.read_csv('data/processed/commodity_data.csv', index_col=0, header=[0,1])
commodity_data.index = pd.to_datetime(commodity_data.index)
commodity_data.tail()

ALIUSD                                               GCUSD  \
               open     high      low    close  volume     vwap    open   
date                                                                      
2025-10-20  2689.25  2689.25  2689.25  2689.25    18.0  2689.25  4269.0   
2025-10-21  2681.25  2681.25  2681.25  2681.25  3894.0  2681.25  4371.0   
2025-10-22  2708.00  2708.00  2708.00  2708.00    44.0  2708.00  4137.0   
2025-10-23  2768.75  2768.75  2768.75  2768.75   162.0  2768.75  4114.8   
2025-10-24  2776.75  2776.75  2776.75  2776.75  2508.0  2776.75  4144.0   

                                    ...  BZUSD                         \
              high     low   close  ...   high    low  close   volume   
date                                ...                                 
2025-10-20  4398.0  4229.7  4359.4  ...  61.55  60.07  61.01  50372.0   
2025-10-21  4393.6  4093.0  4109.1  ...  62.09  60.34  61.32  45150.0   
2025-10-22  4175.0  4021.2  4065.4  ...  63.92  61.40  62.59  46114.0   
2025-10-23  4171.5  4079.6  4145.6  ...  66.34  63.85  65.99  87705.0   
2025-10-24  4159.0  4055.7  4137.8  ...  66.77  65.44  65.94  87705.0   

                    Nickel                        
               vwap   open high low close volume  
date                                              
2025-10-20  61.0150    NaN  NaN NaN   NaN    NaN  
2025-10-21  61.1650    NaN  NaN NaN   NaN    NaN  
2025-10-22  62.3825    NaN  NaN NaN   NaN    NaN  
2025-10-23  65.1075    NaN  NaN NaN   NaN    NaN  
2025-10-24  66.0050    NaN  NaN NaN   NaN    NaN  

[5 rows x 71 columns]

In [51]:
equity_index_data = pd.read_csv('data/processed/index_data.csv', index_col=0, header=[0,1])
equity_index_data.index = pd.to_datetime(equity_index_data.index)
equity_index_data.tail()

^GSPTSE                                                    \
               open     high      low    close       volume      vwap   
date                                                                    
2025-10-20  30265.4  30439.1  30265.4  30416.4  229130400.0  30346.58   
2025-10-21  30214.1  30214.1  29837.6  29888.8  245603000.0  30038.65   
2025-10-22  29818.2  30036.9  29802.5  29983.0  270038700.0  29910.15   
2025-10-23  30097.5  30266.5  30094.9  30186.3  255404100.0  30161.30   
2025-10-24  30248.7  30411.3  30248.7  30353.1  212699600.0  30315.45   

               ^TWII                                ...      ^GDAXI            \
                open      high       low     close  ...      volume      vwap   
date                                                ...                         
2025-10-20  27480.44  27768.27  27412.45  27688.63  ...  46610300.0  24154.24   
2025-10-21  27742.55  27969.05  27742.55  27752.41  ...  42704200.0  24306.50   
2025-10-22  27614.11  27725.83  27490.89  27648.91  ...  57734300.0  24231.93   
2025-10-23  27457.07  27586.85  27371.67  27532.26  ...  51148200.0  24167.15   
2025-10-24       NaN       NaN       NaN       NaN  ...  43268900.0  24234.28   

              ^FCHI                                                  \
               open     high      low    close      volume     vwap   
date                                                                  
2025-10-20  8213.20  8239.67  8151.35  8206.07  62656300.0  8202.57   
2025-10-21  8206.35  8271.48  8198.27  8258.86  58317200.0  8233.74   
2025-10-22  8222.73  8248.90  8193.35  8206.87  61261000.0  8217.96   
2025-10-23  8225.45  8257.47  8215.53  8225.78  66578600.0  8231.06   
2025-10-24  8263.91  8263.91  8169.23  8225.63  55232000.0  8230.67   

           MSCI_China         
                close volume  
date                          
2025-10-20        NaN    NaN  
2025-10-21        NaN    NaN  
2025-10-22        NaN    NaN  
2025-10-23        NaN    NaN  
2025-10-24        NaN    NaN  

[5 rows x 80 columns]

In [52]:
fx_data = pd.read_csv('data/processed/fx_data.csv', index_col=0)
fx_data.index = pd.to_datetime(fx_data.index)
fx_data.tail()

,CADUSD,TWDUSD,AUDUSD,JPYUSD,KRWUSD,EURUSD,GBPUSD,CHFUSD,INRUSD
date,,,,,,,,,
2025-10-20,0.7121,0.032643,0.65122,0.006635,0.000704,1.16413,1.34043,1.2616,0.011363
2025-10-21,0.7128,0.032561,0.64876,0.006581,0.000704,1.16000,1.33683,1.2561,0.011379
2025-10-22,0.7142,0.032471,0.64863,0.006581,0.000699,1.16100,1.33551,1.2560,0.011363
2025-10-23,0.7143,0.032481,0.65106,0.006552,0.000699,1.16181,1.33248,1.2567,0.011397
2025-10-24,0.7141,0.032379,0.65091,0.006540,0.000696,1.16254,1.33099,1.2566,0.011392


In [53]:
def update_level0_with_names(df, symbol_name_dict):
    # Only update if columns are MultiIndex
    if isinstance(df.columns, pd.MultiIndex):
        new_level0 = [f'{symbol_name_dict.get(sym, sym)} ({sym})' for sym in df.columns.get_level_values(0)]
        df.columns = pd.MultiIndex.from_arrays(
            [new_level0] + [df.columns.get_level_values(i) for i in range(1, df.columns.nlevels)],
            names=df.columns.names
        )
    return df

equity_index_data= update_level0_with_names(equity_index_data.copy(), symbol_name_dict)
commodity_data = update_level0_with_names(commodity_data.copy(), symbol_name_dict)

### Converting all Equity Indices to USD
Note: MSCI China Index is already in USD as it was downloaded directly from MSCI's website

In [54]:
# Convert all close prices to USD for indices not already in USD
usd_equity_index_data = equity_index_data.copy()

for idx, row in fmp_idx.iterrows():
    symbol = row['symbol']
    fx_symbol = row['fx_symbol']
    # Only convert if there is a corresponding fx_symbol (i.e., not already in USD)
    if fx_symbol and (symbol, 'close') in usd_equity_index_data.columns and fx_symbol in fx_data.columns:
        fx_series = fx_data[fx_symbol].reindex(usd_equity_index_data.index)
        usd_equity_index_data[(symbol, 'close')] = usd_equity_index_data[(symbol, 'close')] * fx_series

### Computing Derived Data Fields

In [55]:
equity_index_symbols = equity_index_data.droplevel(1, axis=1).columns.unique().to_list()
commodity_symbols = commodity_data.droplevel(1, axis=1).columns.unique().to_list()
all_symbols = equity_index_symbols + commodity_symbols

equity_index_symbols_names_dict = {row['symbol']: row['name'] for idx, row in fmp_idx.iterrows()}
equity_index_symbols_names_dict['MSCI_China'] = 'MSCI China Index'
commodity_symbols_names_dict = {row['symbol']: row['name'] for idx, row in fmp_comm.iterrows()}
commodity_symbols_names_dict['Nickel'] = 'Nickel Futures'
all_symbols_names_dict = {**equity_index_symbols_names_dict, **commodity_symbols_names_dict}

In [56]:
equities_derived = dd.TimeSeriesDerivedFields(price_data=usd_equity_index_data.xs(equity_index_symbols[0], level=0, axis=1)).compute_all_derived_fields()
equities_derived.columns = pd.MultiIndex.from_product([[equity_index_symbols[0]], equities_derived.columns])
for i in range(1, len(equity_index_symbols)):
    temp = dd.TimeSeriesDerivedFields(price_data=usd_equity_index_data.xs(equity_index_symbols[i], level=0, axis=1)).compute_all_derived_fields()
    temp.columns = pd.MultiIndex.from_product([[equity_index_symbols[i]], temp.columns])
    equities_derived = pd.concat([equities_derived, temp], axis=1)

In [57]:
commodities_derived = dd.TimeSeriesDerivedFields(price_data=commodity_data.xs(commodity_symbols[0], level=0, axis=1)).compute_all_derived_fields()
commodities_derived.columns = pd.MultiIndex.from_product([[commodity_symbols[0]], commodities_derived.columns])
for i in range(1, len(commodity_symbols)):
    temp = dd.TimeSeriesDerivedFields(price_data=commodity_data.xs(commodity_symbols[i], level=0, axis=1)).compute_all_derived_fields()
    temp.columns = pd.MultiIndex.from_product([[commodity_symbols[i]], temp.columns])
    commodities_derived = pd.concat([commodities_derived, temp], axis=1)